In [ ]:
!pip install -U diffusers

In [ ]:
pip install -U diffusers transformers accelerate


In [ ]:
!pip install -q torch diffusers transformers accelerate

In [ ]:
"""
import torch
from diffusers import DiffusionPipeline
from diffusers import FluxPipeline
from IPython.display import display

# switch to "mps" for apple devices
# pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", dtype=torch.bfloat16, device_map="cuda")
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload()
def generate(prompt):
  generator = torch.Generator("cpu").manual_seed(42)
  image = pipe(prompt, generator=generator,height=1024,width=1024,num_inference_steps=20).images[0]
  return image
"""

import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from PIL import ImageFilter   # 👈 needed for blur

# 2. Load the Base Model (SDXL 1.0)
base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors"  # 4-step UNet for quality/speed balance

# Load the optimized Lightning UNet
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))

# Create the Pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    base,
    unet=unet,
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# CRITICAL: Use the correct scheduler settings for Lightning
pipe.scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config,
    timestep_spacing="trailing"
)

# ---- SIMPLE SAFETY FILTER (PROMPT) ----
UNSAFE_KEYWORDS = [
    # sexual content
    "sex", "without cloth", "sexual", "nude", "naked", "nsfw", "porn", "boobs",
    "breasts", "nipple", "lingerie", "erotic", "fetish", "strip", "adult content",

    # violent content
    "blood", "gore", "violent", "violence", "kill", "murder", "stab",
    "gun", "knife", "weapon", "shoot", "rape", "abuse"
]

def is_unsafe_prompt(prompt: str) -> bool:
    """
    Returns True if the prompt contains sexual or violent words.
    Very simple keyword-based filter – you can add more words if you want.
    """
    text = (prompt or "").lower()
    # make sure keywords are checked in lowercase
    return any(bad_word.lower() in text for bad_word in UNSAFE_KEYWORDS)


# -------- Unexpected Output Image Checker --------
def is_unsafe_image(image):
    """
    Basic safety check for generated images.
    Heuristic: looks for unusually high red (blood-like) or skin-tone dominance.
    This is a simple rule-based check, not a perfect NSFW detector.
    """
    img = image.convert("RGB")
    pixels = list(img.getdata())

    red_count = 0
    skin_count = 0

    for r, g, b in pixels:
        # Detect blood-like strong red regions
        if r > 180 and g < 80 and b < 80:
            red_count += 1

        # Detect skin-tone like regions (very rough heuristic)
        if (r > 160 and 100 < g < 150 and 80 < b < 130):
            skin_count += 1

    total = len(pixels)

    # thresholds (you can tweak)
    if total == 0:
        return False

    if (red_count / total) > 0.03:     # >3% red dominance = possible gore
        return True
    if (skin_count / total) > 0.10:    # >10% skin-tone = possible nudity
        return True

    return False


# -------- Blur Function (ONLY for unsafe outputs) --------
def blur_image(img, radius: int = 22):
    """
    Apply strong Gaussian blur to unsafe images.
    """
    if img is None:
        return None
    return img.filter(ImageFilter.GaussianBlur(radius))


# 3. Define the Generator Function
def generate_lightning(prompt):
    # 1. Check for unsafe content in the PROMPT first
    if is_unsafe_prompt(prompt):
        # No image will be generated here
        message = "❌ This is violent or sexual content. We cannot do this for you."
        return message, None

    # 2. Safe prompt → generate image normally
    result = pipe(
        prompt,
        num_inference_steps=4,   # Lightning needs exactly 4 steps
        guidance_scale=0         # Lightning requires 0 guidance
    )
    image = result.images[0]

    # 3. Check if the OUTPUT image looks unsafe (unexpected NSFW/violent)
    if is_unsafe_image(image):
        print("⚠️ Unsafe content detected in generated image. Applying blur...")
        image = blur_image(image)   # blur ONLY this unsafe image
        message = "⚠️ Image contained unsafe content and has been blurred."
    else:
        message = "✅ Image generated successfully."

    return message, image




/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:250: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
# 4. Launch the Interface

import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## ⚡ SDXL Lightning (1024px in ~2 Seconds)")

    with gr.Row():
        inp = gr.Textbox(
            label="Enter Prompt",
            placeholder="A cinematic shot of a cyberpunk city..."
        )
        btn = gr.Button("Generate", variant="primary")

    with gr.Row():
        msg_out = gr.Textbox(
            label="Status / Message",
            interactive=False
        )

    img_out = gr.Image(label="Result")

    # NOTE: now outputs is a list: [message, image]
    btn.click(
        fn=generate_lightning,
        inputs=inp,
        outputs=[msg_out, img_out]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68a7848846b9808d80.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
"""
import gradio as gr

with gr.Blocks() as demo:
  gr.Markdown("## Stable Diffusion")
  labelBox = gr.Textbox(label="Enter Prompt",lines=3)
  out = gr.Image()
  gr.Button("Show Image").click(fn=generate,inputs=[labelBox],outputs=[out])
demo.launch()
"""